**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Thu Nov 18 08:54:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
# !ls /mydrive

Mounted at /content/gdrive


**1) Clone the Darknet**



In [4]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15363, done.
remote: Total 15363 (delta 0), reused 0 (delta 0), pack-reused 15363
Receiving objects: 100% (15363/15363), 13.98 MiB | 22.16 MiB/s, done.
Resolving deltas: 100% (10332/10332), done.


**2) Compile Darknet using Nvidia GPU**


In [5]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [6]:
#!cp cfg/yolov3.cfg cfg/yolov3_training.cfg
# !cp /content/gdrive/MyDrive/yolov3/data/yolov3_testing.cfg cfg/yolov3_training.cfg
!cp /mydrive/yolov3/data/yolov3_testing.cfg cfg/yolov3_training.cfg

In [7]:
!echo -e '' > data/obj.names
!echo -e '' > data/obj.data
!cp /mydrive/yolov3/data/obj.names data/obj.names
!cp /mydrive/yolov3/data/obj.data data/obj.data
!mkdir data/obj

In [8]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-11-18 08:57:04--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  41.1MB/s    in 4.2s    

2021-11-18 08:57:09 (37.1 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [9]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
 extracting: data/obj/S2-N1001M00039.jpg  
  inflating: data/obj/S2-N1001M00039.txt  
 extracting: data/obj/S2-N1001M00040.jpg  
 extracting: data/obj/S2-N1001M00040.txt  
  inflating: data/obj/S2-N1001M00041.jpg  
 extracting: data/obj/S2-N1001M00041.txt  
 extracting: data/obj/S2-N1001M00042.jpg  
 extracting: data/obj/S2-N1001M00042.txt  
 extracting: data/obj/S2-N1001M00047.jpg  
  inflating: data/obj/S2-N1001M00047.txt  
 extracting: data/obj/S2-N1001M00048.jpg  
 extracting: data/obj/S2-N1001M00048.txt  
 extracting: data/obj/S2-N1001M00050.jpg  
  inflating: data/obj/S2-N1001M00050.txt  
 extracting: data/obj/S2-N1001M00051.jpg  
  inflating: data/obj/S2-N1001M00051.txt  
 extracting: data/obj/S2-N1001M00052.jpg  
  inflating: data/obj/S2-N1001M00052.txt  
 extracting: data/obj/S2-N1001M00053.jpg  
  inflating: data/obj/S2-N1001M00053.txt  
 extracting: data/obj/S2-N1001M00054.jpg  
  inflating: data/obj/S2-N1001M00054.txt  
 extracting: data

In [12]:
import glob
images_list = sorted(glob.glob("data/obj/*.jpg"))
#images_list = glob.glob("data/obj/*.png")
# print(images_list)

In [17]:
#Create testing.txt file
file = open("data/test.txt", "w") 
file.write("\n".join(images_list[-26:])) 
file.close() 

In [18]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list[:-26])) 
file.close() 

**5) Start the training**

In [13]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.718245), count: 11, class_loss = 0.824034, iou_loss = 0.394120, total_loss = 1.218154 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.784573), count: 11, class_loss = 0.994584, iou_loss = 0.280879, total_loss = 1.275463 
 total_bbox = 154139, rewritten_bbox = 0.116129 % 

 5535: 0.870975, 0.911103 avg loss, 0.000010 rate, 0.386444 seconds, 44280 images, 0.072400 hours left
Loaded: 0.000047 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.785130), count: 3, class_loss = 0.192048, iou_loss = 0.056800, total_loss = 0.248848 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.834231), count: 9, class_loss = 0.254562, iou_loss = 0.098475, total_loss = 0.353038 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.701673), count: 22, class_

In [ ]:
# kick off training from where it last saved
!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_final.weights -dont_show

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolov3_training
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
net.optimized_memory = 0 
mini_batch = 8, batch = 8, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 

In [16]:
# https://keyog.tistory.com/44
# !git clone https://github.com/Cartucho/mAP

Cloning into 'mAP'...
remote: Enumerating objects: 908, done.
remote: Total 908 (delta 0), reused 0 (delta 0), pack-reused 908
Receiving objects: 100% (908/908), 14.71 MiB | 40.60 MiB/s, done.
Resolving deltas: 100% (321/321), done.


In [19]:
# test, mAP 확인
!./darknet detector map data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_final.weights

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
net.optimized_memory = 0 
mini_batch = 1, batch = 1, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut